In [1]:
%jsroot on

In [2]:
TFile* file = TFile::Open("merged_eloss_150k.root");
auto elosstree = (TTree*)file->Get("elosstree");

TVector3* truth_mom = 0;
TVector3 *det_mom = 0;
elosstree->SetBranchAddress("truth_mom", &truth_mom);
elosstree->SetBranchAddress("det_mom", &det_mom);

auto eloss_vs_mom = new TH2F("eloss_vs_mom", "E_loss vs mom ; mom (GeV); E_loss",225,10.,100.,150,6.,12.);
TF1 *swimf = new TF1("swimf","(7.18274+0.0361477*x-0.000718127*pow(x,2)+7.97312e-6*pow(x,3)-3.05481e-8*pow(x,4))",10.0,120.);
//TF1 *atlas_para= new TF1("atlas_para","(5.8117002+0.40835930*log(x)+0.0037488488*x)",10.,100.);


In [3]:
float mu_mass = .105658;
for (int i=0;i<elosstree->GetEntries();i++)
{
    elosstree->GetEntry(i);
    auto intial_energy = sqrt(truth_mom->Mag()*truth_mom->Mag()+mu_mass*mu_mass);
    auto final_energy = sqrt(det_mom->Mag()*det_mom->Mag()+mu_mass*mu_mass);
    auto eloss = intial_energy - final_energy;
    eloss_vs_mom->Fill(truth_mom->Mag(),eloss);
   
}


In [4]:
auto can1 = new TCanvas();
eloss_vs_mom->Draw("colz");
swimf->Draw("same");
//atlas_para_std->Draw("same");
can1->Draw();

In [5]:
 TF1* landaufit=new TF1("landaufit","landau",6.5,9.);
  Double_t peak_eloss[35];
  Double_t peak_eloss_err[35];
  Double_t energy[35];
  Double_t sigma[35];
  Double_t sigma_err[35];
  TCanvas *c3 = new TCanvas("c3","c3");
  c3->Divide(4,4);
  TH1F *projection[35];
  Double_t maxbin,center, width;
  char name[17];
  char title[17];
  for(int i=0;i<17; i++){

    energy[i] = 12.5+5.*i;

    //Int_t lowerbin = dp_p->GetXaxis()->FindBin(5*i+12.5);
    //Int_t upperbin = dp_p->GetXaxis()->FindBin(5*i+17.5); 
    Int_t lowerbin = eloss_vs_mom->GetXaxis()->FindBin(5.*i+10.);
    Int_t upperbin = eloss_vs_mom->GetXaxis()->FindBin(5.*i+15.);

    sprintf(name,"%i<P<%i",5*i+10,5*i+15);

    //projection[i] = (TH1F*)dp_p->ProjectionY(name,lowerbin,upperbin,"");   
    projection[i] = (TH1F*)eloss_vs_mom->ProjectionY(name,lowerbin,upperbin,"");
    //peak_eloss[i]=landaufit->GetParameter(1);
    projection[i]->SetTitle("#DeltaE");
    maxbin = projection[i]->GetMaximumBin();
    center = projection[i]->GetBinCenter(maxbin);
    width = projection[i]->GetRMS();
    c3->cd(i+1);
    projection[i]->Fit("landaufit","Q","",center-width,center+2.*width);
    peak_eloss[i]=landaufit->GetParameter(1);
    peak_eloss_err[i]=landaufit->GetParError(1);
    sigma[i]=landaufit->GetParameter(2);
    sigma_err[i] = landaufit->GetParError(2);
    cout<<"peak value: "<<landaufit->GetParameter(1)<<" and Sigma: "<<landaufit->GetParameter(2)<<endl;
   }


peak value: 7.23988 and Sigma: 0.265861
peak value: 7.36079 and Sigma: 0.293577
peak value: 7.45639 and Sigma: 0.307748
peak value: 7.52592 and Sigma: 0.32711
peak value: 7.58917 and Sigma: 0.344758
peak value: 7.65834 and Sigma: 0.36352
peak value: 7.71468 and Sigma: 0.375684
peak value: 7.75318 and Sigma: 0.388869
peak value: 7.824 and Sigma: 0.422494
peak value: 7.86321 and Sigma: 0.427774
peak value: 7.93849 and Sigma: 0.450964
peak value: 7.99095 and Sigma: 0.46454
peak value: 8.04539 and Sigma: 0.495954
peak value: 8.09307 and Sigma: 0.495677
peak value: 8.12066 and Sigma: 0.530788
peak value: 8.18221 and Sigma: 0.531336
peak value: 8.23181 and Sigma: 0.558203


In [6]:
c3->Draw();

In [11]:
auto can4 = new TCanvas();
TF1 *atlas_para = new TF1("atlas_para","([0]+[1]*log(x)+[2]*x)",5.0,125.);
TGraphErrors* peakE = new TGraphErrors(16,energy,peak_eloss,0,peak_eloss_err);
 //peakE->Draw("AP");
 eloss_vs_mom->Draw("colz");
 peakE->Draw("same");
 peakE->SetMarkerStyle(8);
 peakE->Fit("atlas_para","","",10.,100.);
 gStyle->SetOptFit();

can4->Draw();

 FCN=14.0872 FROM MIGRAD    STATUS=CONVERGED      72 CALLS          73 TOTAL
                     EDM=1.80534e-13    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           6.60990e+00   4.03985e-02   4.88946e-06   2.10525e-04
   2  p1           2.19719e-01   1.63701e-02   1.52483e-06   7.88547e-04
   3  p2           6.64788e-03   4.55060e-04   1.11996e-07   1.01402e-02
